The Unigram algorithm is often used in SentencePiece, which is the tokenization algorithm used by models like AlBERT, T5, mBART, Big Bird, and XLNet.

A Unigram model is a type of language model that considers each token to be independent of the tokens before it. It’s the simplest language model, in the sense that the probability of token X given the previous context is just the probability of token X. So, if we used a Unigram language model to generate text, we would always predict the most common token. Specifically, the probability of a given token is its frequency (the number of times we find it) in the original corpus, divided by the sum of all frequencies of all tokens in the vocabulary (to make sure the probabilities sum up to 1).

At each step of the training, the Unigram algorithm computes a loss over the corpus given the current vocabulary. Then, for each symbol in the vocabulary, the algorithm computes how much the overall loss would increase if the symbol was removed, and looks for the symbols that would increase it the least. Those symbols have a lower effect on the overall loss over the corpus, so in a sense they are “less needed” and are the best candidates for removal.

Each word in the corpus has a score, indeed a probability, and the loss is the negative log likelihood of those scores — that is, the sum for all the words in the corpus of all the -log(P(word)).

In [0]:
corpus = [
    "This info comes from the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [0]:
# we make use of the XLNet tokenizer, which uses Unigram, for the pre-tokenization steps
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

/databricks/python/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [0]:
# count the occurences of each word in the corpus
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'▁This': 3,
             '▁info': 1,
             '▁comes': 1,
             '▁from': 1,
             '▁the': 1,
             '▁Hugging': 1,
             '▁Face': 1,
             '▁Course.': 1,
             '▁chapter': 1,
             '▁is': 1,
             '▁about': 1,
             '▁tokenization.': 1,
             '▁section': 1,
             '▁shows': 1,
             '▁several': 1,
             '▁tokenizer': 1,
             '▁algorithms.': 1,
             '▁Hopefully,': 1,
             '▁you': 1,
             '▁will': 1,
             '▁be': 1,
             '▁able': 1,
             '▁to': 1,
             '▁understand': 1,
             '▁how': 1,
             '▁they': 1,
             '▁are': 1,
             '▁trained': 1,
             '▁and': 1,
             '▁generate': 1,
             '▁tokens.': 1})

In [0]:
# we initialize our vocabulary to be larger than the final size we desire, so we include each individual character (as is necessary for constructing all words in the corpus) and also add the subwords of length at least 2
char_freqs = defaultdict(int)
subwords_freqs = defaultdict(int)
for word, freq in word_freqs.items():
    for i in range(len(word)):
        char_freqs[word[i]] += freq
        # Loop through the subwords of length at least 2
        for j in range(i + 2, len(word) + 1):
            subwords_freqs[word[i:j]] += freq

# Sort subwords by frequency to see preliminarily how many times the most 10 common subwords appear
sorted_subwords = sorted(subwords_freqs.items(), key=lambda x: x[1], reverse=True)
sorted_subwords[:10]

[('▁t', 7),
 ('er', 5),
 ('▁a', 5),
 ('is', 4),
 ('▁to', 4),
 ('to', 4),
 ('en', 4),
 ('▁T', 3),
 ('▁Th', 3),
 ('▁Thi', 3)]

In [0]:
# now we group the characters by frequency, adding the most common subwords, to arrive at an initial vocabulary of 300
# that is, the single characters and the most common subwords
token_freqs = list(char_freqs.items()) + sorted_subwords[: 300 - len(char_freqs)]
token_freqs = {token: freq for token, freq in token_freqs}


NOTE: 💡 SentencePiece uses a more efficient algorithm called Enhanced Suffix Array (ESA) to create the initial vocabulary.

In [0]:
# now we compute the sum of all the frequencies and store the values as the logs of the scores (probabilities of each token) as 
# this is easier computationally when we eventually add them (using log rules) to compute the loss than is multiplying many small floating point numbers
from math import log

total_sum = sum([freq for token, freq in token_freqs.items()])
model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

In [0]:
# This function uses the Viterbi algorithm, computing the score of the segmentations of a word, given a model of the tokens and their frequencies.
# The function, firstly, iteratively fills the 'best_segmentations' list[dict] with segmentations of the word (capturing two keys: its starting point in the word (int) and its segmentation's score (float)) if the current segmentation is found in the vocabulary of tokens and is indeed better than the score of the last segmentation we found in the vocab of tokens with the same ending point in the word.
# We loop this process over the whole string, repeatedly from each character in the string to the ending character in the string and update when necessary. By keeping track of teh starting point of each segmentation we can reconstruct the full tokenization
# The score of the final value in best_segmentations list is the cumulative score of the tokenized word according to its best set of segmentations. We output this value along with the fully, loss maximized, tokenization of the word after all our looping.
def encode_word(word, model):
    best_segmentations = [{"start": 0, "score": 1}] + [
        {"start": None, "score": None} for _ in range(len(word))
    ]
    for start_idx in range(len(word)):
        # This should be properly filled by the previous steps of the loop
        best_score_at_start = best_segmentations[start_idx]["score"]
        for end_idx in range(start_idx + 1, len(word) + 1):
            token = word[start_idx:end_idx]
            if token in model and best_score_at_start is not None:
                score = model[token] + best_score_at_start # keep a running score of the best segmentation as we go through the word
                # If we have found a better segmentation ending at end_idx, we update
                if (
                    best_segmentations[end_idx]["score"] is None
                    or best_segmentations[end_idx]["score"] > score
                ):
                    best_segmentations[end_idx] = {"start": start_idx, "score": score} # update list appropriately if its not yet filled or if the computed cumulative score is greater than the last value from a different starting point

    segmentation = best_segmentations[-1]
    if segmentation["score"] is None:
        # We did not find a tokenization of the word -> unknown
        return ["<unk>"], None

    score = segmentation["score"]
    start = segmentation["start"]
    end = len(word)
    tokens = []
    # iteratively add the tokens to the list of tokens from the end of the word to the beginning as tracked by the best_segmentations list
    while start != 0:
        tokens.insert(0, word[start:end])
        next_start = best_segmentations[start]["start"]
        end = start
        start = next_start
    tokens.insert(0, word[start:end])
    # the score contained here is the cumulative sum of all the best segementations from the beginning to the end of the word
    return tokens, score

In [0]:
# we compute some examples using the encode function
print(encode_word("Hopefully", model))
print(encode_word("This", model))
print(encode_word("hellllllo", model))

(['H', 'o', 'p', 'e', 'f', 'u', 'll', 'y'], 40.349342291887126)
(['This'], 6.311562593298057)
(['he', 'll', 'll', 'll', 'o'], 27.505696965351273)


In [0]:
# now we can compute the total loss of the model based on the frequency of each of its words in the corpus
def compute_loss(model):
    loss = 0
    for word, freq in word_freqs.items():
        _, word_loss = encode_word(word, model)
        loss += freq * word_loss
    return loss

In [0]:
compute_loss(model)

451.5046875675565

In [0]:
# we can reverse engineer our process to calculate the score of each token in the model by removing them and finding the difference in the loss
import copy


def compute_scores(model):
    scores = {}
    model_loss = compute_loss(model)
    for token, score in model.items():
        # We always keep tokens of length 1
        if len(token) == 1:
            continue
        model_without_token = copy.deepcopy(model)
        _ = model_without_token.pop(token)
        scores[token] = compute_loss(model_without_token) - model_loss
    return scores

In [0]:
# we compute the scores and peek at a few tokens
scores = compute_scores(model)
print(scores["ll"])
print(scores["his"])

6.423003528831032
0.0



💡 This approach is very inefficient, so SentencePiece uses an approximation of the loss of the model without token X: instead of starting from scratch, it just replaces token X by its segmentation in the vocabulary that is left. This way, all the scores can be computed at once at the same time as the model loss.

In [0]:
# now we run our training loop
# iterate over the vocab removing words until we reach the desired size
final_vocab_size = 100
percent_to_remove = 0.1
while len(model) > final_vocab_size:
    scores = compute_scores(model)
    sorted_scores = sorted(scores.items(), key=lambda x: x[1])
    # Remove percent_to_remove tokens with the lowest scores.
    for i in range(int(len(model) * percent_to_remove)):
        _ = token_freqs.pop(sorted_scores[i][0])

    total_sum = sum([freq for token, freq in token_freqs.items()])
    model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

In [0]:
def tokenize(text, model):
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in words_with_offsets]
    encoded_words = [encode_word(word, model)[0] for word in pre_tokenized_text]
    return sum(encoded_words, [])


tokenize("This info comes from the Hugging Face section about tokenization.", model)

['▁This',
 '▁info',
 '▁comes',
 '▁from',
 '▁the',
 '▁Hugging',
 '▁Face',
 '▁section',
 '▁about',
 '▁tokenization.']